In [ ]:
# Import important libraries
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import gdown
import os

In [ ]:
# Constants
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

In [ ]:
# Google Drive file_ids of commercial and non-commercial audio-video features
commercialFeatures_150_file_id          = "1ucT-jrkmhRHx8RSMW0bueGD2IjL3ZpiO"
nonCommercialFeatures_150_file_id       = "1uA5sdu_5b2l0z8k_aI0NeG2eqWsdX9zi"

# Destination paths
commercialFeatures_150_destination      = "commercialFeatures_150.pkl"
nonCommercialFeatures_150_destination   = "nonCommercialFeatures_150.pkl"

In [ ]:
def download_file_from_google_drive(file_id, destination):
    """
    Download a file from Google Drive if it doesn't already exist locally.

    Parameters:
    - file_id (str): The unique file ID from Google Drive.
    - destination (str): The local path where the file should be saved.
    """
    if os.path.exists(destination):
        print(f'File already exists at {destination}. Skipping download.')
    else:
        url = f"https://drive.google.com/uc?id={file_id}"
        gdown.download(url, destination, quiet=False)

In [ ]:
# Downloading commercialFeatures
download_file_from_google_drive(commercialFeatures_150_file_id, commercialFeatures_150_destination)

# Downloading nonCommercialFeatures
download_file_from_google_drive(nonCommercialFeatures_150_file_id, nonCommercialFeatures_150_destination)

File already exists at commercialFeatures_150.pkl. Skipping download.
File already exists at nonCommercialFeatures_150.pkl. Skipping download.


In [ ]:
def load_features():
    """
    Load data from a pickled file if it exists, otherwise initialize an empty list.

    Returns:
    - commercialFeatures_150 (list) - List containing Commercial video features
    - nonCommercialFeatures_150 (list) - List containing nonCommercial video features
    """
    with open('commercialFeatures_150.pkl', 'rb') as f:
        commercialFeatures_150 = pickle.load(f)
    with open('nonCommercialFeatures_150.pkl', 'rb') as f:
        nonCommercialFeatures_150 = pickle.load(f)

    print(f'Number of commercial features: {len(commercialFeatures_150)}')
    print(f'Number of nonCommercial features: {len(nonCommercialFeatures_150)}')
    print("Audio-Visual Features Loaded Successfully")
    return commercialFeatures_150, nonCommercialFeatures_150

In [ ]:
commercialFeatures_150, nonCommercialFeatures_150 = load_features()

Number of commercial features: 4364
Number of nonCommercial features: 4597
Audio-Visual Features Loaded Successfully


In [ ]:
print(commercialFeatures_150.shape)

(4364, 150, 1152)


In [ ]:
print(nonCommercialFeatures_150.shape)

(4597, 150, 1152)


# Generating labels for data and merging commercial and non-commercial features and shuffling data randomly

In [ ]:
# Generating labels
commercial_labels = np.ones(len(commercialFeatures_150))
non_commercial_labels = np.zeros(len(nonCommercialFeatures_150))

In [ ]:
# Merging commercial and non-commercial features
X = np.concatenate((commercialFeatures_150, nonCommercialFeatures_150), axis=0)
Y = np.concatenate((commercial_labels, non_commercial_labels), axis=0)

In [ ]:
print(f'Shape of data is {X.shape}')
print(f'Shape of labels is {Y.shape}')

Shape of data is (8961, 150, 1152)
Shape of labels is (8961,)


In [ ]:
# Shuffling data randomly
X, Y = shuffle(X, Y, random_state=42)

# Preprocess the data

In [ ]:
print(X[0][0])

[219.  78. 255. ... 124. 141. 177.]


In [ ]:
# Finding Range of data
maxi = 0
mini = 1000
for i in range(len(X[0][0])):
    maxi = max(maxi, X[0][0][i])
    mini = min(mini, X[0][0][i])

In [ ]:
print(f'maximum value is {maxi}')
print(f'minimum value is {mini}')

maximum value is 255.0
minimum value is 0.0


In [ ]:
def scale_data(data):
    """
    Scale the input data by dividing by 255.

    Parameters:
    data (numpy.ndarray): Input data array of shape [samples, features] or higher dimensions.

    Returns:
    numpy.ndarray: Scaled data with values in the range [0, 1].
    """

    scaled_data = data / 255.0
    return scaled_data

In [ ]:
X = scale_data(X)

In [ ]:
print(X[0][0])

[0.85882353 0.30588235 1.         ... 0.48627451 0.55294118 0.69411765]


In [ ]:
def standardize_data(data):
    """
    Standardize the input data.

    Parameters:
    data (numpy.ndarray): Input data array of shape [samples, features] or higher dimensions.
                          If the input has more than 2 dimensions, the function will flatten the
                          data along the features axis, standardize, and then reshape back.

    Returns:
    numpy.ndarray: Standardized data with mean 0 and standard deviation 1 along the features axis.
    """

    original_shape = data.shape
    if len(original_shape) > 2:
        data = data.reshape(-1, original_shape[-1])

    scaler = StandardScaler()
    standardized_data = scaler.fit_transform(data)

    if len(original_shape) > 2:
        standardized_data = standardized_data.reshape(original_shape)

    return standardized_data

In [ ]:
X = standardize_data(X)

In [ ]:
print(X[0][0])

[ 1.55840786 -0.51457659  3.16766892 ...  0.02947117  0.20901008
  0.56678746]


# Train, Test and Validation split

In [ ]:
def split_data(data, labels, train_ratio=0.7, valid_ratio=0.15, test_ratio=0.15, random_state=None):
    """
    Split data into training, validation, and test sets.

    Parameters:
    data (numpy.ndarray): The input data of shape [samples, features].
    labels (numpy.ndarray): The corresponding labels of shape [samples,].
    train_ratio (float): The ratio of the training set. Default is 0.7.
    valid_ratio (float): The ratio of the validation set. Default is 0.15.
    test_ratio (float): The ratio of the test set. Default is 0.15.
    random_state (int, optional): Random seed for reproducibility.

    Returns:
    tuple: Tuple containing:
        - train_data (numpy.ndarray): Training data.
        - valid_data (numpy.ndarray): Validation data.
        - test_data (numpy.ndarray): Test data.
        - train_labels (numpy.ndarray): Training labels.
        - valid_labels (numpy.ndarray): Validation labels.
        - test_labels (numpy.ndarray): Test labels.
    """

    assert train_ratio + valid_ratio + test_ratio == 1, "Ratios must sum to 1."

    train_data, temp_data, train_labels, temp_labels = train_test_split(
        data, labels, test_size=(1 - train_ratio), random_state=random_state
    )

    temp_ratio = valid_ratio + test_ratio
    valid_ratio_relative = valid_ratio / temp_ratio
    test_ratio_relative = test_ratio / temp_ratio

    valid_data, test_data, valid_labels, test_labels = train_test_split(
        temp_data, temp_labels, test_size=test_ratio_relative, random_state=random_state
    )

    return train_data, valid_data, test_data, train_labels, valid_labels, test_labels

In [ ]:
train_data, valid_data, test_data, train_labels, valid_labels, test_labels = split_data(X, Y, random_state=42)

In [ ]:
print(f'Train data shape: {train_data.shape}')
print(f'Validation data shape: {valid_data.shape}')
print(f'Test data shape: {test_data.shape}')

Train data shape: (6272, 150, 1152)
Validation data shape: (1344, 150, 1152)
Test data shape: (1345, 150, 1152)


In [ ]:
print(f'Train labels shape: {train_labels.shape}')
print(f'Validation labels shape: {valid_labels.shape}')
print(f'Test labels shape: {test_labels.shape}')

Train labels shape: (6272,)
Validation labels shape: (1344,)
Test labels shape: (1345,)


# Building a model for Classification



# Model Training



# Evaluating model on Test data

# Exporting the trained model